In [40]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
import matplotlib.animation as animation
%matplotlib notebook

# Advection Equation
In this notebook I will be solving the following advection equation
$$ \frac{\partial u}{\partial t} + v \frac{\partial u}{\partial x} = 0 $$
with the initial and boundary conditions given as
$$ u(x,0) = I(x), \qquad u(0,t) = U_0 $$ 

## Leapfrog Method
We use the following discretization procedure.
$$ [D_{2t}u + vD_{2x}u = 0]_i^n $$
which will result in 
$$  \frac{u_i^{n+1} - u_i^{n-1}}{2\Delta t} + v \frac{u_{i+1}^n - u_{i-1}^n}{2\Delta x} = 0. $$ 
By orginizing the known and unkown terms we will get
$$  u_{i}^{n+1} = u_{i}^{n-1} - C(u_{i+1}^n - u_{i-1}^n)  $$ 
where $ C $ is the Courant number and is given as $ C = v \Delta t / \Delta x $.
And for the first time step we use the following scheme
$$ u_{i}^{1} = u_{i}^{0} - \frac12 C(u_{i+1}^0 - u_{i-1}^0) $$

In [225]:
def applyDirichlet(x,value):
    x[0] = value
    return x

def u_0(x):
    sigma = 0.05
    u = np.exp(-0.5*((x-L/4)/sigma)**2)
    u = applyDirichlet(u,0)
    return u

def update(frame):
    line.set_ydata(uList[:, 15*frame])
    return [line]



In [235]:
L = 1
T = 1
v = 1
C = 1
## Change to 0.05 to see some oscilation following the wave
## Change to 1 to see a convergent run, but still has problem when hiting the boundary
## Cangge to 1.01 to see some unstable solution
dt = 0.001
dx = v*dt/C

xList = np.arange(0,L,dx)
tList = np.arange(0,T,dt)
uList = np.zeros((xList.shape[0],tList.shape[0]))


In [236]:
uList[:,0] = u_0(xList)

## First time step update
uList[1:-1,1] = uList[1:-1,0] - 1/2*C*(uList[2:,0] - uList[0:-2,0])  ## interior points
uList[0,1] = uList[0,0] - 1/2*C*(uList[1,0] - uList[-1,0]) ## periodic boundary condition 
uList[-1,1] = uList[-1,0] - 1/2*C*(uList[0,0] - uList[-2,0]) ## periodic boundary condition

# uList[-1,1] = uList[-1,0]  -C*(uList[0,0] - uList[-1,0])
# uList[0,1] = uList[0,0]  -C*(uList[1,0] - uList[0,0])  ## to apply the Dirichlet boundary condition

for n in range(2,tList.shape[0]):
    ## Discrete equation for interior point
    uList[1:-1,n] = uList[1:-1:,n-2] - C*(uList[2:,n-1]-uList[0:-2,n-1])  

    ## periodic boundary condition
    uList[-1,n] = uList[-1,n-2] - C*(uList[0,n-1]-uList[-2,n-1]) 
    uList[0,n] = uList[0,n-2] - C*(uList[1,n-1]-uList[-1,n-1]) 


In [237]:
fig, ax = plt.subplots(figsize=(10,5))
# Initialize the plot
line, = ax.plot(xList, uList[:, 0])
ani = animation.FuncAnimation(fig, update, frames=60, interval=1, blit=True)
# Display the animation in the notebook
HTML(ani.to_jshtml())


<IPython.core.display.Javascript object>

## Upwind method
Consider the advection equation below
$$ \frac{\partial u}{\partial t} + v \frac{\partial u}{\partial x} = 0 $$
For $v>0$ we will have advection to the right, so in a upwind method we do the differentiation agains the flow. For instance, for $v > 0$ we will use the following approximation
$$ [D^+_t u + v D^-_x u = 0]_i^n $$
or in the exapnded form
$$ \frac{u_i^{n+1} - u_i^n}{\Delta t} + v\frac{u^n_i - u^n_{i-1}}{\Delta x} = 0. $$
By collecting the known and unknown terms we will have
$$ u_i^{n+1} = u_i^n - C(u^n_i - u^n_{i-1}) $$

For $ C \leq 1$ this scheme is stable. It turns out that upwind method is the same as doing forward difference in time and center difference in space of $ u_t + v u_x = \alpha u_xx $ where $ \alpha = v\Delta x/2 $. In summary, FTCS is unstable when we have pure advection, but becomes stable once we add some artificial diffusion.

In [247]:
L = 1
T = 1
v = 1
C = 1
## Change to 0.05. Oscilation is gone but the wave spreads out as if there is diffusion term
## Change to 1 to see a convergent run, but still has problem when hiting the boundary
## Cangge to 1.05 to see some unstable solution
dt = 0.001
dx = v*dt/C

xList = np.arange(0,L,dx)
tList = np.arange(0,T,dt)
uList = np.zeros((xList.shape[0],tList.shape[0]))

In [248]:
uList[:,0] = u_0(xList)
for n in range(1,tList.shape[0]):
    uList[1:,n] = uList[1:,n-1] - C*(uList[1:,n-1]-uList[0:-1,n-1])
    uList[0,n] = uList[0,n-1] - C*(uList[0,n-1]-uList[-1,n-1])

In [249]:
fig, ax = plt.subplots(figsize=(10,5))
# Initialize the plot
line, = ax.plot(xList, uList[:, 0])
ani = animation.FuncAnimation(fig, update, frames=60, interval=1, blit=True)
# Display the animation in the notebook
HTML(ani.to_jshtml())


<IPython.core.display.Javascript object>